<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados.</font>
<br>
<font size='1'> Modificado en 2017-2, 2018-1, 2018-2, 2019-1, 2019-2, 2020-1, 2020-2 y 2022-2 por Equipo Docente IIC2233</font>
</p>

# Tabla de contenidos

1. [Levantando excepciones: `raise`](#Levantando-excepciones:-raise)
2. [Manejo de Excepciones: `try` y `except`](#Manejo-de-Excepciones:-try-y-except)

# Levantando excepciones: `raise`

Hemos visto algunos tipos de excepciones existentes y las condiciones bajo las cuales se gatillan.

Sin embargo, también podemos generar una excepción en el momento que queramos creando una nueva instancia de la excepción, y utilizando la sentencia **`raise`**. Cada excepción tiene un tipo definido, y es posible definir un mensaje explicativo para el usuario.

Por ejemplo, la siguiente función `convertir_coordenada` recibe un *string* que sigue el formato `"coordenada_x, coordenada_y"` y tiene por objetivo retornar una tupla con los valores enteros de la coordenada. Se podría lanzar excepciones de esta manera:

In [1]:
def convertir_coordenada(coordenada_en_string):
    # Si el input no es del tipo esperado
    if not isinstance(coordenada_en_string, str):
        # aquí se genera una excepción y se incluye información para el usuario.
        raise TypeError("Coordenada debe ser un string")

    # Si el input no sigue el formato esperado
    if "," not in coordenada_en_string:
        # aquí se genera otra excepción y se incluye información para el usuario.
        raise ValueError("Coordenada debe estar separada por una coma")

    coord_x, coord_y = coordenada_en_string.split(",")
    return (int(coord_x), int(coord_y))

In [2]:
texto_1 = "56, 1123"
x, y = convertir_coordenada(texto_1)
print(x, y)

56 1123


In [3]:
no_texto = [43, 3]
x, y = convertir_coordenada(no_texto)
print(x, y)

TypeError: Coordenada debe ser un string

In [4]:
texto_incorrecto = "23 - 54"
x, y = convertir_coordenada(texto_incorrecto)
print(x, y)

ValueError: Coordenada debe estar separada por una coma

Las excepciones interrumpen todo el flujo del programa. Aún si la excepción ocurre dentro de una llamada a una función que ha sido llamada dentro de otras llamadas a función, todas las llamadas a funciones previas son interrumpidas y la excepción llega hasta el sistema operativo (a menos que sea capturada previamente como veremos más adelante). Notar que en los últimos dos ejemplos no se ejecutó el `print`, debido a que se interrumpió el programa antes.

A continuación, puedes ver otro ejemplo, donde al instanciar un objeto podemos exigir que el inicializador tenga que recibir una tupla de dos elementos como parámetro de entrada, lanzando una excepción cuando recibimos un argumento de otro tipo.

In [5]:
class Circulo:

    def __init__(self, centro):
        if not isinstance(centro, tuple):
            raise TypeError("El parámetro 'centro' debe ser de tipo 'tuple'")
            # Recordar que cuando ocurre el raise, la ejecución se interrumpe.
            print("Esta línea nunca se imprime")
        self.centro = centro

    def __repr__(self):
        return f"El centro es {self.centro}"

In [6]:
# Caso correcto donde se crea un objeto usando una tupla.
c1 = Circulo((2,3))
c1

El centro es (2, 3)

In [7]:
# Caso que genera la excepción al usar una lista como entrada.
c2 = Circulo([2,3])
c2

TypeError: El parámetro 'centro' debe ser de tipo 'tuple'

# Manejo de Excepciones: `try` y `except`

Cada vez que se **levanta** una excepción durante la ejecución del código, es posible **atraparla** mediante el uso de las sentencias `try` y `except`.

La sentencia `try` permite definir un *scope* (bloque de código). Si se levanta una excepción dentro del *scope* de `try`, entonces la excepción es **capturada**. A continuación del bloque de `try` debe haber una o más instrucciones `except`. Las instrucciones `except` permiten implementar el manejo de la excepción capturada.

En el momento que se captura una excepción dentro de `try` el flujo del programa salta inmediatamente al bloque de una de las sentencias `except`. Una vez que el bloque `except` ha terminado, el programa continúa en la instrucción **posterior** al bloque `try`/`except`. El programa **NO regresa** a la sentencia que gatilló la excepción.

Cómo se mencionó al inicio de esta sección, solo se atrapan excepciones que surgen durante la ejecución del código. Esto implica que excepciones del tipo `SyntaxError` o `IndentationError` no son posible de **atrapar** porque estas surgen durante la lectura del programa, no su ejecución.

Volvamos al caso de la función `dividir`:

In [8]:
# No se atrapa ninguna excepción, por lo tanto no se previene un posible error.
def dividir(num, den):
    # Esta función terminará el programa cuando el
    # denominador den sea 0.
    return float(num) / float(den)

In [9]:
r = 4
w = dividir(5, r)
print(w)
z = dividir(5, r - 4)
print(z)
print("Ya hice ambas divisiones.")

1.25


ZeroDivisionError: float division by zero

Primero probaremos "protegiendo" un llamado correcto a `dividir`, usando un bloque `try`/`except`.

In [10]:
r = 4
try:
    # Dentro de este bloque ejecutamos el código que PODRÍA
    # arrojar una excepción.
    print(dividir(5, r))
    print("Ya hice la división.")

except (ZeroDivisionError) as error:
    # Aquí manejamos la excepción que pueda ser lanzada en
    # el bloque anterior. Si un error del tipo ZeroDivisonError
    # ocurre, se ejecuta este bloque y el resto del programa
    # continúa su ejecución normal. La excepción, como objeto,
    # se puede acceder con la variable error.
    print(f"Error: {error}")
    print("¿Cómo se te ocurre dividir por cero? ¿Por qué eres así?")

print("El programa continúa después del try/except")

1.25
Ya hice la división.
El programa continúa después del try/except


En este caso no se generó la excepción, por lo tanto, no se ejecutó el código dentro de `except` y el programa siguió su flujo normal.

Ahora probaremos protegiendo un caso en el que los argumentos de la división genera la excepción, ya que el denominador es 0.

In [11]:
# Manejo de la excepción con argumentos inválidos.
# En este caso la función dará un error debido a que el denominador
# utilizado es 0.
r = 4
try:
    print(dividir(5, r))
    print(dividir(5, r - 4))
    print("Ya hice ambas divisiones.")

except ZeroDivisionError as error:
    print(f"Error: {error}")
    print("¿Cómo se te ocurre dividir por cero? ¿Por qué eres así?")

print("El programa continúa después del try/except")

1.25
Error: float division by zero
¿Cómo se te ocurre dividir por cero? ¿Por qué eres así?
El programa continúa después del try/except


Podemos observar que cuando se produjo la excepción dentro del llamado `dividir(5,r - 4)`, se generó una excepción de tipo `ZeroDivisionError`. Como esta excepción no fue capturada dentro de `dividir`, entonces la excepción se propagó al llamado anterior, pero este llamado sí estaba dentro de un `try`. El flujo del programa salta hacia la sentencia `except` que está asociada al tipo de la excepción (`ZeroDivisionError`) y ejecuta lo que hay dentro de ella. Una vez que se ha ejecutado todo lo que había dentro de `except ZeroDivisionError`, el flujo del programa continúa normalmente.

Pongamos atención a dos cosas: (1) la instrucción `print("Ya hice ambas divisiones")` nunca se ejecuta porque al capturar la excepción, el flujo salta inmediatamente al `except`; (2) como la excepción fue exitosamente manejada, una vez que el `except` se completó, el programa siguió su flujo normal y, a diferencia de los ejemplos anteriores, no se "cayó".

Cómo se mencionó al inicio de esta sección, solo se atrapan excepciones que surgen durante la ejecución del código. Esto implica que excepciones del tipo `SyntaxError` o `IndentationError` no son posible de **atrapar** en el mismo archivo porque estas surgirán durante la lectura del programa, no su ejecución.

In [12]:
# Atrapar SyntaxError en el mismo código --> Fallará.
try:
    print("Anya
except SyntaxError:
    print("SyntaxError detectado")

SyntaxError: unterminated string literal (detected at line 3) (589727527.py, line 3)

No obstante, igual existen formas de atrapar estos errores. Una de ellas es utilizar `import` de un posible archivo con errores de sintaxis o indentación, y dejar el `import archivo` dentro de un `try/excepy`. Esta solución logrará atrapar este tipo de error porque durante la ejecución del `import` es que recién se intentará leer el archivo con `SyntaxError`.

En el siguiente ejemplo se dejó el mismo código con error `print("Anya` en un archivo llamado `archivo_con_syntax_error.py` y ahora se hará `import` de él, pero atrapando la excepción de `SyntaxError`.

In [13]:
# Atrapar SyntaxError desde un import --> Funcionará.
try:
    import archivo_con_syntax_error
except SyntaxError:
    print("SyntaxError detectado")

SyntaxError detectado


### Múltiples excepciones con `except`

Con la sentencia `except` podemos incluir varios tipos de excepciones para manejar. En el siguiente caso, las causas que generarán las excepciones son distintas, pero el tratamiento para todas ellas será el mismo. Si lo que necesitamos es realizar un tratamiento diferenciado según el tipo de excepción, entonces podemos añadir más bloques de `except`, en donde cada uno tiene definido las excepciones sobre las que debe actuar.

In [14]:
def convertir_coordenada(coordenada_en_string):
    # Verificamos que el input es del tipo esperado.
    if not isinstance(coordenada_en_string, str):
        # Intencionalmente levantamos la excepción. Recuerda que las excepciones son objetos.
        raise TypeError("Coordenada debe ser un string")

    # Verificamos que el input sigue el formato esperado.
    if "," not in coordenada_en_string:
        # El mensaje incluido en la excepción es el que se despliega
        # cuando la manejamos después.
        raise ValueError("Coordenada debe estar separada por una coma")

    coord_x, coord_y = coordenada_en_string.split(",")
    return (int(coord_x), int(coord_y))

Ahora manejamos la excepción que pueda ser lanzada durante la ejecución de la función `convertir_coordenada`.

El primer caso levantará una excepción debido a que el argumento no es válido al no ser del tipo esperado (no es de tipo `str`).

In [15]:
try:
    print(convertir_coordenada([42, 3]))

# En esta parte manejamos las excepciones una vez que son lanzadas.
except TypeError as error:
    # Este bloque sólo maneja excepciones del tipo TypeError.
    print(f"Error: {error}")
    print("Revise el tipo del argumento.")

except ValueError as error:
    # Este bloque sólo maneja excepciones del tipo ValueError.
    print(f"Error: {error}")
    print("Se produjo un ValueError. Verifique sus valores.")

Error: Coordenada debe ser un string
Revise el tipo del argumento.


En este segundo caso, se levantará una excepción porque el argumento no sigue el formato esperado.

In [16]:
try:
    print(convertir_coordenada("32 - 190"))

# En esta parte manejamos las excepciones una vez que son lanzadas.
except TypeError as error:
    # Este bloque sólo maneja excepciones del tipo TypeError.
    print(f"Error: {error}")
    print("Revise el tipo del argumento.")

except ValueError as error:
    # Este bloque sólo maneja excepciones del tipo ValueError.
    print(f"Error: {error}")
    print("Se produjo un ValueError. Verifique sus valores.")

Error: Coordenada debe estar separada por una coma
Se produjo un ValueError. Verifique sus valores.


También, es posible capturar varios tipos de errores con un solo bloque `except`. Se logra entregando una tupla de tipos de excepción en vez de solamente un tipo. Como para el siguiente ejemplo de una función dividir personalizada, que agrupa los errores `ZeroDivisionError` y `TypeError`.

In [17]:
def dividir(num, den):
    if not (isinstance(num, int) and isinstance(den, int)):
        raise TypeError("Error de tipo en numerador o denominador. :'(")

    if num < 0 or den < 0:
        raise ValueError("Hay un valor negativo entre numerador y denominador >:(")

    return float(num)/float(den)

In [18]:
for argumentos in [(45, "5"), (-23, 5), (4, 0), (21, 7)]:
    try:
        print(f"Llamada a dividir{argumentos}")
        print(dividir(*argumentos))

    # En esta parte manejamos las excepciones una vez que son lanzadas.
    except (ZeroDivisionError, TypeError) as error:
        # Este bloque maneja excepciones del tipo ZeroDivisionError o TypeError.
        print(f"Error: {error}")
        print("Revise los datos de la división. Hay un tipo incorrecto.\n")

    except ValueError as error:
        # Este bloque sólo maneja excepciones del tipo ValueError.
        print(f"Error: {error}")
        print("Se produjo un ValueError. Verifique sus valores.\n")

Llamada a dividir(45, '5')
Error: Error de tipo en numerador o denominador. :'(
Revise los datos de la división. Hay un tipo incorrecto.

Llamada a dividir(-23, 5)
Error: Hay un valor negativo entre numerador y denominador >:(
Se produjo un ValueError. Verifique sus valores.

Llamada a dividir(4, 0)
Error: float division by zero
Revise los datos de la división. Hay un tipo incorrecto.

Llamada a dividir(21, 7)
3.0


### Flujos complementarios: `else` y `finally`

El bloque de `try` y `except` puede ser complementado opcionalmente con las sentencias **`else`** y **`finally`**:

- Las instrucciones dentro del bloque `else` se ejecutarán **siempre y cuando no se haya lanzado ninguna excepción**.
- En el bloque de la sentencia `finally` van instrucciones que se realizan **siempre, independientemente de si ocurrió una excepción o no**.

In [19]:
# Esta corresponde a la estructura completa de try and except.
try:
    # Probamos si es posible realizar la operación.
    resultado = dividir(10, 0)
    print("Esta línea no se ejecuta si se produce una excepción en la línea anterior.")

except (ZeroDivisionError, TypeError):
    # Este bloque opera para los tipos de excepciones definidos.
    print("Revise los datos de entrada. ¡No son int o bien el denominador es 0!")

except ValueError:
    # Este bloque sólo maneja excepciones del tipo ValueError.
    print("Los valores ingresados son negativos")

else:
    # Como no hubo excepciones puede retornar normalmente el resultado
    # En este caso, si se coloca un return después de la operación y
    # esta es correcta, entonces nunca llegará a este punto.
    print("¡Todo OK!, no hay errores con los datos")

finally:
    print("Recuerde SIEMPRE usar excepciones para manejar los errores de su programa")

Revise los datos de entrada. ¡No son int o bien el denominador es 0!
Recuerde SIEMPRE usar excepciones para manejar los errores de su programa


En el ejemplo, si no se levanta ninguna excepción, también sería impreso el mensaje en la sección **`else`**. El siguiente ejemplo ilustra eso:

In [20]:
# Esta corresponde a la estructura completa de try and except.
try:
    # Probamos si es posible realizar la operación.
    resultado = dividir(10, 5)
    print("Esta línea no se ejecuta si se produce una excepción en la línea anterior.")

except (ZeroDivisionError, TypeError):
    # Este bloque opera para los tipos de excepciones definidos.
    print("Revise los datos de entrada. ¡No son int o bien el denominador es 0!")

except ValueError:
    # Este bloque sólo maneja excepciones del tipo ValueError.
    print("Los valores ingresados son negativos")

else:
    # Como no hubo excepciones puede retornar normalmente el resultado
    # En este caso, si se coloca un return después de la operación y
    # esta es correcta, entonces nunca llegará a este punto.
    print("¡Todo OK!, no hay errores con los datos")

finally:
    print("Recuerde SIEMPRE usar excepciones para manejar los errores de su programa")

Esta línea no se ejecuta si se produce una excepción en la línea anterior.
¡Todo OK!, no hay errores con los datos
Recuerde SIEMPRE usar excepciones para manejar los errores de su programa


El uso de **`finally`** es común en la ejecución de funciones de limpieza predefinidas, como por ejemplo, el cierre de un archivo después de ser procesado. Si ocurren errores mientras el archivo está abierto, éste quedará abierto. Es importante crear una rutina que asegure que, independientemente de si se lanza o no alguna excepción, el archivo sea cerrado correctamente.

In [21]:
fid = open("log.txt", "w")

try:
    # Probamos si es posible realizar la apertura del archivo.
    # En este caso se debe generar un error por que el denominador llega a ser 0.
    for i in range(5, -1, -1):
        fid.write(f"{dividir(10, i)}")

except (ZeroDivisionError, TypeError):
    # Este bloque opera para los tipos de excepciones definidos.
    print("¡Error!: Revise los datos de entrada ¡No son ints o bien el denominador es 0!")

else:
    print("El archivo fue creado correctamente!")

finally:
    # Este bloque asegura que el archivo sea cerrado correctamente
    # independientemente de si se produjo el error.
    print("Recuerde SIEMPRE cerrar sus archivos")
    fid.close()

¡Error!: Revise los datos de entrada ¡No son ints o bien el denominador es 0!
Recuerde SIEMPRE cerrar sus archivos


Una forma equivalente de hacer esto mismo es usar un *context manager* mediante la sentencia **`with`**. El siguiente ejemplo resume esta manera:

In [22]:
with open("log.txt", "w") as fid:
    try:
        # Probamos si es posible realizar la apertura del archivo.
        # En este caso se debe generar un error por que el denominador llega a ser 0.
        for i in range(5, -1, -1):
            fid.write(f"{dividir(10, i)}")

    except (ZeroDivisionError, TypeError):
        # Este bloque opera para los tipos de excepciones definidos.
        print("¡Error!: Revise los datos de entrada ¡No son ints o bien el denominador es 0!")

¡Error!: Revise los datos de entrada ¡No son ints o bien el denominador es 0!


## Observaciones

El manejo de excepciones es otra forma de control del flujo del programa, similar a lo que ocurre con la sentencia `if`. Entonces, ¿qué conviene más usar? ¿`if`/`else` o excepciones?. **Usar excepciones es más recomendable** que usar `if`/`elif`/`else` **para controlar errores**. Siempre es posible crear un sistema de códigos de error manejado por distintas salidas (`return`) de una función o módulo y suficientes combinaciones de `if`/`elif`/`else`. Sin embargo, esto puede generar casos particulares que complejizan, ensucian el diseño, y le quitan flexibilidad a nuestro programa. Además, continuamente hay que estar agregando nuevos códigos de error, lo que dificulta la mantenibilidad de nuestro código. 

En general, las principales ventajas de usar excepciones por sobre `if`/`else` son:
- El programador está obligado a darles algún tratamiento, es decir, manejarlas o levantarlas. Mientras que los códigos de error pueden ser ignorados por el programador.
- El código queda más limpio y fácil de leer (recuerden que el código se lee muchas más veces de lo que se escribe).
- Todas las situaciones del programa son manejadas genéricamente, mientras que usando códigos de error tenemos la obligación de crear estructuras de control para cada función que implementemos.
- El manejo de excepciones permite "notificar" a otras aplicaciones sobre este tipo de situaciones, lo que no sería tan simple de lograr usando códigos de error inventados por el programador.
- ¿Por qué importa que el programa no falle inesperadamente?: Muchas veces exponer errores que no se han manejado a usuarios finales puede ser peligroso, ya que se podrían visualizar trozos de código en los outputs de estos. 

**Revisa la sección 2 de los ejercicios propuestos para aplicar el levantamiento y manejo de excepciones en tu código.**